## TP1, Ejercicio 4

Dado que el que mejor se comporto, fue el Linear SVM, voy a partir de alli. Primero reveo su resultado

In [2]:
%run tagging/scripts/eval.py -c ancora-3.0.1es -i baselineSVM -m


Accuracy: 94.11% / 97.57% / 62.76% (total / known / unk)

g \ m	sp000	nc0s000	da0000	aq0000	fc	nc0p000	rg	np00000	fp	cc
sp000	14.30	0.00	-	0.03	-	-	0.00	-	-	-	
nc0s000	0.00	12.09	0.01	0.33	-	0.02	0.02	0.08	-	0.00	
da0000	-	0.08	9.53	-	-	0.00	-	0.00	-	-	
aq0000	0.00	0.34	-	6.61	-	0.15	0.01	0.05	-	-	
fc	-	-	-	-	5.85	-	-	-	-	-	
nc0p000	-	0.25	-	0.31	-	4.90	-	0.04	-	-	
rg	0.02	0.02	0.00	0.21	-	0.01	3.35	0.01	-	0.02	
np00000	0.00	0.21	-	0.08	-	0.01	-	3.26	-	0.00	
fp	-	-	-	-	-	-	-	-	3.55	-	
cc	0.00	-	-	0.01	-	-	0.05	0.00	-	3.34	


En eval.py ya estamos calculando algunas variables utiles para el analisis:

- **error_count:** matriz de confusión completa.
- **error_sents:** matriz de índices de las oraciones en las que ocurre cada error.

Por ejemplo, podemos ver algunas que tienen los mas altos indices de error

Adjetivo con sustantivo comun en singular

In [13]:
error_count['aq0000']['nc0s000'] 

319

In [8]:
sorted(error_sents['aq0000']['nc0s000'])[:10]

[32, 50, 70, 77, 112, 122, 127, 129, 135, 164]

In [11]:
sent = sents[32]
words, tags = zip(*sent)  # separar las palabras de los tags
sent[:10]

[('La', 'da0000'),
 ('empresa', 'nc0s000'),
 ('canadiense', 'aq0000'),
 ('Alcan_Aluminium', 'np00000'),
 ('anunció', 'vmis000'),
 ('el', 'da0000'),
 ('jueves', 'da0000'),
 ('que', 'cs'),
 ('ha', 'vaip000'),
 ('llegado', 'vmp0000')]

La inversa

In [14]:
error_count['nc0s000']['aq0000']  

312

Adjetivo con sustantivo comun en plural

In [15]:
error_count['nc0p000']['aq0000']  

290

Nombre propio con sustantivo comun

In [6]:
error_count['np00000']['nc0s000'] 

202

Curioso es lo siguiente, sustantivos comunes en singular y plural

In [7]:
error_count['nc0p000']['nc0s000'] 

241

Este ultimo, aunque no del mayor error, pareceria el mas sensillo de resolver. Solo identificando si termina o no con s, puede solucionarnos el problema. Probaremos esto para la ventana de 3 palabras con que se trabajaba


In [42]:
%run tagging/scripts/train.py -c ancora-3.0.1es -m class -k svm -o baselineSVMs

In [44]:
%run tagging/scripts/eval.py -c ancora-3.0.1es -m -i baselineSVMs


Accuracy: 94.47% / 97.57% / 66.37% (total / known / unk)

g \ m	sp000	nc0s000	da0000	aq0000	fc	nc0p000	rg	np00000	fp	cc
sp000	14.30	0.01	-	0.02	-	0.00	0.00	-	-	-	
nc0s000	0.00	12.14	0.01	0.29	-	0.01	0.02	0.08	-	0.00	
da0000	-	0.07	9.53	0.00	-	-	-	0.01	-	-	
aq0000	0.01	0.35	-	6.53	-	0.24	0.01	0.05	-	-	
fc	-	-	-	-	5.85	-	-	-	-	-	
nc0p000	-	0.01	-	0.20	-	5.28	-	0.02	-	-	
rg	0.02	0.03	0.00	0.19	-	0.02	3.35	0.01	-	0.02	
np00000	0.00	0.20	-	0.06	-	0.02	-	3.28	-	0.00	
fp	-	-	-	-	-	-	-	-	3.55	-	
cc	0.00	0.00	-	0.00	-	0.01	0.05	0.00	-	3.34	


In [45]:
error_count['nc0p000']['nc0s000'] 

6

Vemos que se redujo considerablemente este error. 

Veamos que sucede con el nombre propio confundido con el sustantivo comun

In [48]:
error_count['np00000']['nc0s000'] 

187

In [49]:
sorted(error_sents['np00000']['nc0s000'])[:10]

[4, 6, 9, 12, 29, 36, 37, 47, 61, 68]

In [50]:
sent = sents[12]
words, tags = zip(*sent)  # separar las palabras de los tags
sent[:20]

[('Para', 'sp000'),
 ('el', 'da0000'),
 ('director', 'nc0s000'),
 ('general', 'aq0000'),
 ('de', 'sp000'),
 ('Investigación_de_la_CAM', 'np00000'),
 (',', 'fc'),
 ('Francisco_José_Rubia', 'np00000'),
 (',', 'fc'),
 ('es', 'vsip000'),
 ('necesario', 'aq0000'),
 ('apoyar', 'vmn0000'),
 ('a_fondo', 'rg'),
 ('la', 'da0000'),
 ('investigación', 'nc0s000'),
 ('de', 'sp000'),
 ('calidad', 'nc0s000'),
 ('e', 'cc'),
 ('incentivar', 'vmn0000'),
 ('con', 'sp000')]

Como varios de estos contienen un guion bajo, agrego una feature para intentar ayudar a discriminarlo

In [51]:
%run tagging/scripts/train.py -c ancora-3.0.1es -m class -k svm -o baselineSVMsu

In [52]:
%run tagging/scripts/eval.py -c ancora-3.0.1es -m -i baselineSVMsu


Accuracy: 94.47% / 97.57% / 66.37% (total / known / unk)

g \ m	sp000	nc0s000	da0000	aq0000	fc	nc0p000	rg	np00000	fp	cc
sp000	14.30	0.01	-	0.02	-	0.00	0.00	-	-	-	
nc0s000	0.00	12.14	0.01	0.29	-	0.01	0.02	0.08	-	0.00	
da0000	-	0.07	9.53	0.00	-	-	-	0.01	-	-	
aq0000	0.01	0.35	-	6.53	-	0.24	0.01	0.05	-	-	
fc	-	-	-	-	5.85	-	-	-	-	-	
nc0p000	-	0.01	-	0.20	-	5.28	-	0.02	-	-	
rg	0.02	0.03	0.00	0.19	-	0.02	3.35	0.01	-	0.02	
np00000	0.00	0.20	-	0.06	-	0.02	-	3.28	-	0.00	
fp	-	-	-	-	-	-	-	-	3.55	-	
cc	0.00	0.00	-	0.00	-	0.01	0.05	0.00	-	3.34	


In [53]:
error_count['np00000']['nc0s000'] 

187

no hay mejoras...aunque probare aplicar los mismos cambios a lr, que era el segundo mejor

In [57]:
%run tagging/scripts/train.py -c ancora-3.0.1es -m class -k lr -o baselineLRsu

In [60]:
%run tagging/scripts/eval.py -c ancora-3.0.1es -m -i baselinelr


Accuracy: 91.69% / 95.01% / 61.66% (total / known / unk)

g \ m	sp000	nc0s000	da0000	aq0000	fc	nc0p000	rg	np00000	fp	cc
sp000	14.26	0.01	-	0.05	-	-	0.01	0.00	-	-	
nc0s000	0.00	11.89	0.01	0.52	-	0.02	0.02	0.09	-	-	
da0000	-	0.11	9.50	0.01	-	0.01	-	0.01	-	-	
aq0000	0.00	0.51	-	6.43	-	0.17	0.01	0.06	-	-	
fc	-	-	-	-	5.85	-	-	-	-	-	
nc0p000	-	0.35	-	0.47	-	4.61	-	0.04	-	-	
rg	0.02	0.04	0.00	0.39	-	0.00	3.11	0.03	-	0.02	
np00000	-	0.22	-	0.08	-	0.01	-	3.24	-	0.00	
fp	-	-	-	-	-	-	-	-	3.55	-	
cc	0.00	0.00	-	0.01	-	0.00	0.05	0.00	-	3.34	


In [59]:
%run tagging/scripts/eval.py -c ancora-3.0.1es -m -i baselineLRsu


Accuracy: 92.38% / 95.42% / 64.85% (total / known / unk)

g \ m	sp000	nc0s000	da0000	aq0000	fc	nc0p000	rg	np00000	fp	cc
sp000	14.27	0.01	-	0.04	-	0.00	0.01	0.00	-	-	
nc0s000	0.00	12.02	0.01	0.41	-	0.01	0.01	0.09	-	-	
da0000	-	0.11	9.50	0.01	-	-	-	0.01	-	-	
aq0000	0.00	0.56	-	6.21	-	0.35	0.01	0.06	-	-	
fc	-	-	-	-	5.85	-	-	-	-	-	
nc0p000	-	0.01	-	0.22	-	5.26	-	0.02	-	-	
rg	0.02	0.05	0.00	0.32	-	0.03	3.14	0.03	-	0.02	
np00000	-	0.20	-	0.07	-	0.03	-	3.26	-	0.00	
fp	-	-	-	-	-	-	-	-	3.55	-	
cc	0.00	0.00	-	0.00	-	0.01	0.05	0.00	-	3.34	


In [61]:
%run tagging/scripts/train.py -c ancora-3.0.1es -m class -k svm -o baselineSVMsul

In [62]:
%run tagging/scripts/eval.py -c ancora-3.0.1es -m -i baselineSVMsul


Accuracy: 94.47% / 97.57% / 66.37% (total / known / unk)

g \ m	sp000	nc0s000	da0000	aq0000	fc	nc0p000	rg	np00000	fp	cc
sp000	14.30	0.01	-	0.02	-	0.00	0.00	-	-	-	
nc0s000	0.00	12.14	0.01	0.29	-	0.01	0.02	0.08	-	0.00	
da0000	-	0.07	9.53	0.00	-	-	-	0.01	-	-	
aq0000	0.01	0.35	-	6.53	-	0.24	0.01	0.05	-	-	
fc	-	-	-	-	5.85	-	-	-	-	-	
nc0p000	-	0.01	-	0.20	-	5.28	-	0.02	-	-	
rg	0.02	0.03	0.00	0.19	-	0.02	3.35	0.01	-	0.02	
np00000	0.00	0.20	-	0.06	-	0.02	-	3.28	-	0.00	
fp	-	-	-	-	-	-	-	-	3.55	-	
cc	0.00	0.00	-	0.00	-	0.01	0.05	0.00	-	3.34	


In [63]:
error_count['np00000']['nc0s000'] 

187

Tampco noto ventajas por sobre lo anterior. Otras ideas, viendo los casos donde fallan bastandome en los error counts iniciales y buscando en internet, tenian que ver con intentar capturar prefijos y subfijos. Como -ito, -ato, -oso, -mente, a- , hiper-, etc... tambien otros caracteres especiales como dieresis o tildes. Lo primero seria mas interesante usando un diccionario con estos fragmentos, aunque creo que no es la idea del ejercicio implementar todas las ideas. La segunda es mas sensilla de implementar, asi que la probare...

In [64]:
%run tagging/scripts/train.py -c ancora-3.0.1es -m class -k svm -o baselineSVMsule

In [65]:
%run tagging/scripts/eval.py -c ancora-3.0.1es -m -i baselineSVMsule


Accuracy: 94.47% / 97.57% / 66.37% (total / known / unk)

g \ m	sp000	nc0s000	da0000	aq0000	fc	nc0p000	rg	np00000	fp	cc
sp000	14.30	0.01	-	0.02	-	0.00	0.00	-	-	-	
nc0s000	0.00	12.14	0.01	0.29	-	0.01	0.02	0.08	-	0.00	
da0000	-	0.07	9.53	0.00	-	-	-	0.01	-	-	
aq0000	0.01	0.35	-	6.53	-	0.24	0.01	0.05	-	-	
fc	-	-	-	-	5.85	-	-	-	-	-	
nc0p000	-	0.01	-	0.20	-	5.28	-	0.02	-	-	
rg	0.02	0.03	0.00	0.19	-	0.02	3.35	0.01	-	0.02	
np00000	0.00	0.20	-	0.06	-	0.02	-	3.28	-	0.00	
fp	-	-	-	-	-	-	-	-	3.55	-	
cc	0.00	0.00	-	0.00	-	0.01	0.05	0.00	-	3.34	
